In [6]:
import os
import requests
import threading
import urllib.request, urllib.error, urllib.parse
import time

url = "https://apod.nasa.gov/apod/image/1901/LOmbradellaTerraFinazzi.jpg"

In [7]:
# fungsi mensplit  data yang akan di download
def buildRange(value, numsplits):
    # mendifine array lst
    lst = []
    # perulangan dari i hingga numsplits
    for i in range(numsplits):
        # jika i == 0 atau iterasi pertama
        if i == 0:
            lst.append('%s-%s' % (i, int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))))
        # iterasi berikutnya
        else:
            lst.append('%s-%s' % (int(round(1 + i * value/(numsplits*1.0),0)), int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))))
    return lst

In [8]:
# memisahkan buffer sesuai dengan jumlah thread yang ada
class SplitBufferThreads(threading.Thread):
    """ Splits the buffer to ny number of threads
        thereby, concurrently downloading through
        ny number of threads.
    """
    def __init__(self, url, byteRange):
        super(SplitBufferThreads, self).__init__()
        self.__url = url
        self.__byteRange = byteRange
        self.req = None

    def run(self):
        self.req = urllib.request.Request(self.__url,  headers={'Range': 'bytes=%s' % self.__byteRange})

    def getFileData(self):
        return urllib.request.urlopen(self.req).read()

In [9]:
# main function dilakukan split sebanyak 3 buah
def main(url=None, splitBy=3):
    #waktu mulai
    start_time = time.time()
    
    # jika url yang dimasukan tidak valid
    if not url:
        print("Please Enter some url to begin download.")
        return
    
    # menamai file dengan url setelah slash terakhir
    fileName = url.split('/')[-1]
    
    # menghitung size dalam byte pada gambar
    sizeInBytes = requests.head(url, headers={'Accept-Encoding': 'identity'}).headers.get('content-length', None)
    # tampilkan sizeInBytes nya
    print("%s bytes to download." % sizeInBytes)
    
    # jika bukan byte yang disimpan
    if not sizeInBytes:
        print("Size cannot be determined.")
        return
    
    # mendifine array dataLst
    dataLst = []
    # melakukan perulangan dari i sampai splitBy=3
    for idx in range(splitBy):
        # menentukan byte range dari gambar yang akan di download
        byteRange = buildRange(int(sizeInBytes), splitBy)[idx]
        
        # mensplit buffer thread berdasarkan url dan byteRange
        bufTh = SplitBufferThreads(url, byteRange)
        
        # start buffer
        bufTh.start()
        
        #join buffer
        bufTh.join()
        
        #memasukan file yang akan di download ke dataLst
        dataLst.append(bufTh.getFileData())

    content = b''.join(dataLst)
    # cek apakah dataLst berisi data
    if dataLst:
        # cek keberadaan file dengan menggunakan os
        if os.path.exists(fileName):
            #remove file dari os
            os.remove(fileName)
        # tampilkan jangka waktu download
        print("--- %s seconds ---" % str(time.time() - start_time))
        # write file yang sudah di download ke dalam komputer
        with open(fileName, 'wb') as fh:
            fh.write(content)
        # tampilkan bahwa berhasi me write file
        print("Finished Writing file %s" % fileName)

In [10]:
#panggil fungsi utama
if __name__ == '__main__':
    main(url)

3670260 bytes to download.
--- 34.71994137763977 seconds ---
Finished Writing file LOmbradellaTerraFinazzi.jpg
